In [1]:
import pandas as pd
import datetime
import numpy as np
import warnings

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif']=['SimHei']       # 中文
plt.rcParams['axes.unicode_minus'] = False       # 负号
path = "D:/program/program1/User_portrait/"

In [2]:
dot_data = pd.read_csv(path + "20190311.csv", names = ['dot', 'order_date', 'company', 'order_num'])
dot_data['order_date'] = pd.to_datetime(dot_data['order_date'], format = '%Y-%m-%d')

In [3]:
dot_info = pd.read_csv(path + 'dot_1.csv', header = None, names = ['dot', 'check_date', 'dot_type', 'city_code', 'area', 'province', 'city', 'district', 'town', 'village'])

In [4]:
dot_data.sort_values(by = ['company', 'order_date'], inplace = True)
dot_data = dot_data[dot_data['order_date'] >= pd.to_datetime('2017-02-08')]

In [5]:
len(dot_data['dot'].unique())

6167

In [6]:
dot_data.head()

,dot,order_date,company,order_num
5144302,虹口七浦点部,2018-12-28,01个人刘健,3
4221130,虹口七浦点部,2018-12-30,01个人刘健,2
4939912,园区胜浦点部,2017-09-15,3M苏州,1
8064199,园区保税区点部,2017-09-16,3M苏州,1
10325546,园区保税区点部,2017-09-18,3M苏州,1


In [7]:
# freq_week = pd.date_range('2017-02-08', '2019-02-08', freq = 'w')
# freq_week

In [8]:
data = dot_data[dot_data['dot'].isin(dot_info['dot'])]
data.reset_index(drop = True, inplace = True)

In [9]:
len(data['dot'].unique())

2414

In [10]:
data.head()

,dot,order_date,company,order_num
0,虹口七浦点部,2018-12-28,01个人刘健,3
1,虹口七浦点部,2018-12-30,01个人刘健,2
2,园区胜浦点部,2017-09-15,3M苏州,1
3,园区保税区点部,2017-09-16,3M苏州,1
4,园区保税区点部,2017-09-18,3M苏州,1


日活跃客户数

In [19]:
day_active = pd.DataFrame(data.groupby(by = ['dot', 'order_date'])['company'].count())
day_active.reset_index(inplace = True)
day_active.head()

,dot,order_date,company
0,三水乐平点部,2018-05-17,5
1,三水乐平点部,2018-05-18,1
2,三水乐平点部,2018-05-24,1
3,三水乐平点部,2018-05-28,2
4,三水乐平点部,2018-05-30,1


In [22]:
tmp = data.copy()
tmp['dayofweek'] = tmp.order_date.dt.dayofweek
tmp.head()

,dot,order_date,company,order_num,dayofweek
order_date,,,,,
2018-12-28,虹口七浦点部,2018-12-28,01个人刘健,3,4
2018-12-30,虹口七浦点部,2018-12-30,01个人刘健,2,6
2017-09-15,园区胜浦点部,2017-09-15,3M苏州,1,4
2017-09-16,园区保税区点部,2017-09-16,3M苏州,1,5
2017-09-18,园区保税区点部,2017-09-18,3M苏州,1,0


In [32]:
dayofweek = pd.DataFrame(tmp.groupby(['dot', 'dayofweek'])['company'].count()).reset_index()

点部日运单量的最大值，最小值，平均值

In [12]:
day_active.groupby('dot')['company'].agg({'max':'max', 'min':'min', 'avg':'mean'}).head()

,max,min,avg
dot,,,
三水乐平点部,26,1,11.980159
三水木棉点部,20,1,9.326446
三水竹山京东点部,5,1,1.904762
三水竹山点部,22,1,10.896694
上虞上浦点部,22,1,8.578049


周活跃客户数

In [13]:
data.index = data['order_date']

In [14]:
data.head()

,dot,order_date,company,order_num
order_date,,,,
2018-12-28,虹口七浦点部,2018-12-28,01个人刘健,3
2018-12-30,虹口七浦点部,2018-12-30,01个人刘健,2
2017-09-15,园区胜浦点部,2017-09-15,3M苏州,1
2017-09-16,园区保税区点部,2017-09-16,3M苏州,1
2017-09-18,园区保税区点部,2017-09-18,3M苏州,1


周活跃数

In [15]:
week_active = pd.DataFrame(data.groupby('dot').resample('W')['company'].count()).reset_index()
week_active.head()

,dot,order_date,company
0,三水乐平点部,2018-05-20,6
1,三水乐平点部,2018-05-27,1
2,三水乐平点部,2018-06-03,6
3,三水乐平点部,2018-06-10,76
4,三水乐平点部,2018-06-17,103


两周活跃数

In [16]:
pd.DataFrame(data.groupby('dot').resample('2W')['company'].count()).reset_index().head()

,dot,order_date,company
0,三水乐平点部,2018-05-20,6
1,三水乐平点部,2018-06-03,7
2,三水乐平点部,2018-06-17,179
3,三水乐平点部,2018-07-01,159
4,三水乐平点部,2018-07-15,146


月活跃数

In [17]:
pd.DataFrame(data.groupby('dot').resample('M')['company'].count()).reset_index().head()

,dot,order_date,company
0,三水乐平点部,2018-05-31,10
1,三水乐平点部,2018-06-30,336
2,三水乐平点部,2018-07-31,298
3,三水乐平点部,2018-08-31,336
4,三水乐平点部,2018-09-30,347


客户活跃度：活跃用户, 不活跃用户，流失用户

In [11]:
dot_info.head()

,dot,check_date,dot_type,city_code,area,province,city,district,town,village
0,清江新下塘点部,2018-09-13,大点部,577,华东,浙江省,温州市,乐清市,城东街道,新下塘村
1,香洲洪湾点部,2018-08-13,大点部,756,华南,广东省,珠海市,香洲区,南屏镇,洪湾社区
2,贵阳三甫点部,2018-05-15,大点部,851,华北,贵州省,贵阳市,观山湖区,金华镇,三甫村
3,镇江大港点部,2018-05-10,大点部,511,华东,江苏省,镇江市,镇江新区,大路镇,大路村
4,三水竹山点部,2018-06-06,大点部,757,华南,广东省,佛山市,三水区,乐平镇,竹山村
